### Installazione delle dipendenze

Richiede Python3.10 o superiore. Dovresti creare un virtual environment con il comando:
```bash
python -m venv .venv
```

Dopodichè puoi eseguire la cella successiva dentro al virtual environment.

In [55]:
!python --version
!pip install -r requirements.txt

Python 3.10.4


You should consider upgrading via the 'C:\Users\tonto\OneDrive\Desktop\magistrale\bd2\projects\cap\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [56]:
import pandas as pd
import numpy as np

### Definizione costanti

In [59]:
DATA_PATH = "pg_stat_statements_16_20_60s.csv"
CALLS = [30, 29]

### Importazione statistiche da file CSV

In [60]:
df = pd.read_csv(DATA_PATH)
df.head()

,userid,dbid,toplevel,queryid,query,plans,total_plan_time,min_plan_time,max_plan_time,mean_plan_time,...,wal_fpi,wal_bytes,jit_functions,jit_generation_time,jit_inlining_count,jit_inlining_time,jit_optimization_count,jit_optimization_time,jit_emission_count,jit_emission_time
0,10,16419,True,-3825640688184167069,SELECT * FROM pg_stat_database,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,16419,True,1345865735253463468,SELECT * FROM pg_stat_user_tables,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,16419,True,3545412574037053608,SET DateStyle=ISO,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,16419,True,-6911109443227572258,SELECT * FROM pg_stat_bgwriter,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,16419,True,-141619989335985508,SELECT * FROM pg_stat_archiver,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df.columns

Index(['userid', 'dbid', 'toplevel', 'queryid', 'query', 'plans',
       'total_plan_time', 'min_plan_time', 'max_plan_time', 'mean_plan_time',
       'stddev_plan_time', 'calls', 'total_exec_time', 'min_exec_time',
       'max_exec_time', 'mean_exec_time', 'stddev_exec_time', 'rows',
       'shared_blks_hit', 'shared_blks_read', 'shared_blks_dirtied',
       'shared_blks_written', 'local_blks_hit', 'local_blks_read',
       'local_blks_dirtied', 'local_blks_written', 'temp_blks_read',
       'temp_blks_written', 'blk_read_time', 'blk_write_time',
       'temp_blk_read_time', 'temp_blk_write_time', 'wal_records', 'wal_fpi',
       'wal_bytes', 'jit_functions', 'jit_generation_time',
       'jit_inlining_count', 'jit_inlining_time', 'jit_optimization_count',
       'jit_optimization_time', 'jit_emission_count', 'jit_emission_time'],
      dtype='object')

### Eliminazione delle colonne che non ci servono

In [62]:
columns = ["query", "calls", "total_exec_time", "min_exec_time", "max_exec_time", "mean_exec_time", "stddev_exec_time", "blk_read_time", "blk_write_time"]
df = df[columns]
df = df[df["calls"].isin(CALLS)]

### Calcolo delle variabili operazionali

Aggiustiamo il tempo di risposta rimuovendo i valori massimo e minimo, considerandoli outliers.

Calcoliamo il service demand di CPU e disco.

In [63]:
df["mean_blk_read_time"] = df["blk_read_time"] / df["calls"]
df["mean_blk_write_time"] = df["blk_write_time"] / df["calls"]
df["adj_mean_exec_time"] = (df["total_exec_time"] - df["min_exec_time"] - df["max_exec_time"]) / (df["calls"] - 2)
df["D_CPU"] = df["adj_mean_exec_time"] - df["mean_blk_read_time"] - df["mean_blk_write_time"]
df["D_DISK"] = df["mean_blk_read_time"] + df["mean_blk_write_time"]
df["CPU_vs_DISK"] = df["D_CPU"] / df["D_DISK"]

In [64]:
df

,query,calls,total_exec_time,min_exec_time,max_exec_time,mean_exec_time,stddev_exec_time,blk_read_time,blk_write_time,mean_blk_read_time,mean_blk_write_time,adj_mean_exec_time,D_CPU,D_DISK,CPU_vs_DISK
6,"SELECT\n p_brand,\n p_type,\n p_size,\n ...",30,26965.064068,862.053529,1401.235346,898.835469,94.622963,430.789520,0,14.359651,0.0,882.206257,867.846606,14.359651,60.436471
22,"select s_name, s_address from supplier, nation...",29,35798.147510,999.819857,2774.755912,1234.418880,317.583333,23278.985471,0,802.723637,0.0,1186.058213,383.334576,802.723637,0.477542


### Query CPU-Intensive

In [68]:
q_cpu = df.loc[df["CPU_vs_DISK"].idxmax()]
print(q_cpu)

query                  SELECT\n   p_brand,\n   p_type,\n   p_size,\n ...
calls                                                                 30
total_exec_time                                             26965.064068
min_exec_time                                                 862.053529
max_exec_time                                                1401.235346
mean_exec_time                                                898.835469
stddev_exec_time                                               94.622963
blk_read_time                                                  430.78952
blk_write_time                                                         0
mean_blk_read_time                                             14.359651
mean_blk_write_time                                                  0.0
adj_mean_exec_time                                            882.206257
D_CPU                                                         867.846606
D_DISK                                             

### Query Disk-Intensive

In [69]:
q_disk = df.loc[df["CPU_vs_DISK"].idxmin()]
print(q_disk)

query                  select s_name, s_address from supplier, nation...
calls                                                                 29
total_exec_time                                              35798.14751
min_exec_time                                                 999.819857
max_exec_time                                                2774.755912
mean_exec_time                                                1234.41888
stddev_exec_time                                              317.583333
blk_read_time                                               23278.985471
blk_write_time                                                         0
mean_blk_read_time                                            802.723637
mean_blk_write_time                                                  0.0
adj_mean_exec_time                                           1186.058213
D_CPU                                                         383.334576
D_DISK                                             